In [2]:
import numpy as np
import matplotlib.pyplot as plt 
from datetime import datetime 

In [4]:
from util import get_data

## Information Entolopy  
With binary class `y==0 or 1`, definition of informaiton entropy is as follows.   
$H(y)=-p(y==0)log_2p(y==0)-p(y==1)log_2p(y==1)$

In [13]:
# y to be 1D numpy array 
def entropy(y):
    # y = 0 or 1 
    N = len(y) 
    s1 = (y==1).sum() 
    if 0 == s1 or N == s1:
        return 0  # no variation, entropy is zero 
    p1 = float(s1)/N 
    p0 = 1- p1 
    return -p0*np.log2(p0) - p1*np.log2(p1)

In [16]:
# Quick test of entropy()
print(entropy(np.array([0,0,0,1,1,1])))
print(entropy(np.array([0,0,0,0,0,0])))
print(entropy(np.array([0,0,1,1,1,1])))

1.0
0
0.9182958340544896


## Information Gain  
Both `x and y` are 1D array.   
Given a threshold value to `x` as `split`, calculate information gain from the split.  
$IG(x,y\mid{split}) = 1 - p(x<split)H(y\mid{x<split}) - p(x\geq{split})H(y\mid{x\geq{split}})$ 

In [64]:
# x,y to be 1D numpy array with same length (x=input variable, y=target class(0 or 1))
# split is threshold value to x to make split 
def information_gain(x, y, split): 
    y0 = y[x < split]
    y1 = y[x >= split]
    N = len(y) 
    y0len = len(y0)
    if y0len==0 or y0len==N: 
        return 0    # already H(y)=0, and result is always 0 even if calculated 
    p0 = float(len(y0))/N 
    p1 = 1 - p0 
    return entropy(y) - p0*entropy(y0) - p1*entropy(y1)

In [66]:
# Quick test of information_gain() 
x = np.array([0,0,0,0,0,0]); y = np.array([0,0,0,0,0,0]); split=0.5
print( information_gain(x,y,split) )

x = np.array([1,1,1,0,0,0]); y = np.array([1,1,1,0,0,0]); split=0.5
print( information_gain(x,y,split) )

x = np.array([1.,2.,3.,7.,8.,9.]); y = np.array([1.,1.,1.,0.,0.,0.]); split=5.0
print( information_gain(x,y,split) )

0
1.0
1.0


## Find Best Split that gives Maximum Information Gain for a specified feature  
`X={NxD}` input training data and `Y={N,1}` target training data.  `Y` is either 0 or 1.  
`col` is column index to `X`.  `X[:,col]` is extracted at the beginning of thsi function.   
Then, both `x_values=X[:,col]` and `Y` are sorted in ascending order of `x_values`. 
With sorted `Y`, the boundary of different class is examined as caididate points to calculate infomration gain.   
Threshold for potential `split` is calculated as midpoint of `x_values` over the boundary position.  
Return the maximum information gain found, and the `split` value used for the maximum's case.  
If this `col` gives maximum information gain w.r.t. the other columns in X, then both `col` and `split` is kept as information for the decision tree node for prediction.  

In [107]:
# quick confirmation of np.nonzero() usage 
y_values = np.array([0,0,1,1,0,0])
boundaries = np.nonzero(y_values[:-1] != y_values[1:])

# np.nonzero(a) return tuple of arrays, one for each dimension of a. 
# In this case, input is 1D, so we're interested only in 1st element of tupple. 
print(boundaries[0]) # [1 3] is expectd 

# example from numpy doc. 
# (row,col) for non-zero is (0,0), (1,1), (2,0), (2,1)
x = np.array([[1,0,0], 
              [0,2,0], 
              [1,1,0]])
print(x)
y = np.nonzero(x) 
print(y)
print(y[0])  # row index      [0 1 2 2]
print(y[1])  # column index   [0 1 0 1]

[1 3]
[[1 0 0]
 [0 2 0]
 [1 1 0]]
(array([0, 1, 2, 2], dtype=int64), array([0, 1, 0, 1], dtype=int64))
[0 1 2 2]
[0 1 0 1]


In [71]:
def find_split(X, Y, col): 
    x_values = X[:,col]
    sort_idx = np.argsort(x_values)  # index of the sorted 
    x_values = x_values[sort_idx]
    y_values = Y[sort_idx] 
    
    #print(x_values)
    #print(y_values)
    
    # get boundary of different target y 
    boundaries = np.nonzero(y_values[:-1] != y_values[1:])[0]
    #print(boundaries)
    
    best_split = None 
    max_ig = 0 
    for b in boundaries: 
        # set split to middle point of boundary 
        split = (x_values[b] + x_values[b+1]) / 2 
        # calculate information gain 
        ig = information_gain(x_values, y_values, split) 
        if ig > max_ig: 
            max_ig = ig
            best_split = split 
    
    return max_ig, best_split

In [72]:
# simple test of find_split() 
X = np.array([[2,3,4,8,9,7]]).T
Y = np.array([1,1,1,0,0,0])
print(find_split( X, Y, 0 ))

(1.0, 5.5)


## TreeNode class   
For simplicity, this handles only binary label with values either 0 or 1.  

In [111]:
class TreeNode: 
    
    # class variable for maximum depth 
    max_depth_used = 0
    
    def __init__(self, depth=0, max_depth=None):
        self.depth = depth 
        self.max_depth = max_depth 
        if self.depth == 0: 
            TreeNode.max_depth_used = 0
        elif depth > TreeNode.max_depth_used:
            TreeNode.max_depth_used = depth
        
    def fit(self, X, Y):
        if len(Y)==1 or len(set(Y))==1: 
            # already single class, i.e., leaf node 
            self.col = None
            self.split = None 
            self.left = None 
            self.right = None 
            self.prediction = Y[0]
        else:
            D = X.shape[1]
            cols = range(D) 
                        
            # find the column whose separation gives maximum information gain 
            max_ig = 0 
            best_col = None 
            best_split = None 
            for col in cols:
                ig, split = find_split(X, Y, col) 
                if ig > max_ig: 
                    max_ig = ig
                    best_col = col
                    best_split = split
                    
            # maximum information gain == 0, then no way to split any more
            # use majority voting as prediction value 
            if max_ig == 0: 
                # col=None & split=None : prediction single value 
                self.col = None
                self.split = None 
                self.left = None 
                self.right = None 
                self.prediction = np.round(Y.mean())
            else:
                self.col = best_col
                self.split = best_split 
                
                # check against max_depth 
                # if already at max_depth, prepare 2 prediction values based on majority after split 
                if self.depth == self.max_depth: 
                    self.left = None 
                    self.right = None 
                    self.prediction = [ 
                        np.round( Y[X[:,best_col] < self.split].mean() ),
                        np.round( Y[X[:,best_col] >= self.split].mean() ),
                    ]
                else:
                    # split X,Y based on split and make recursive 
                    left_idx = (X[:,best_col]<best_split)
                    right_idx = (X[:,best_col]>=best_split)
                                        
                    Xleft = X[left_idx] ;   Yleft = Y[left_idx]    
                    self.left = TreeNode( self.depth+1, self.max_depth ) 
                    self.left.fit(Xleft, Yleft)
                    
                    Xright = X[right_idx];  Yright = Y[right_idx]
                    self.right = TreeNode( self.depth+1, self.max_depth ) 
                    self.right.fit(Xright, Yright) 
        return
                    
    def predict_one(self, x):
        if self.col is not None and self.split is not None:
            feature = x[self.col]
            if feature < self.split:
                if self.left: 
                    # recursion 
                    p = self.left.predict_one(x) 
                else:
                    # pre-allocated calss as left 
                    p = self.prediction[0]
            else:
                if self.right:
                    # recursion 
                    p = self.right.predict_one(x)
                else:
                    # pre-allocated class as right 
                    p = self.prediction[1]
        else:
            # pre-allocated class for leaf node 
            p = self.prediction 
        return p
    
    def predict(self, X):
        N = len(X) # row size 
        P = np.zeros(N) 
        for i in range(N): 
            P[i] = self.predict_one(X[i])
        return P
                
            

## Wrapper class  
Just to appear as if sklern... 

In [112]:
class DecisionTree: 
    def __init__(self, max_depth=None):
        self.max_depth = max_depth 
        
    def fit(self, X,  Y):
        self.root = TreeNode(max_depth=self.max_depth)
        self.root.fit(X,Y)
        self.max_depth_used_ = TreeNode.max_depth_used
        
    def predict(self,X):
        return self.root.predict(X)
    
    def score(self, X, Y): 
        P = self.predict(X) 
        return np.mean(P==Y) 
    

## Run Test 

In [83]:
# Load MNIST data 
Xall,Yall = get_data() 

Reading in and transforming data...


In [86]:
# For this binary dicision tree, use only label==0 & ==1 
idx = np.logical_or(Yall==0, Yall==1) 
X = Xall[idx]
Y = Yall[idx]
X.shape, Y.shape

((8816, 784), (8816,))

In [92]:
# Train and Test data split
Ntrain = len(Y) // 2
Xtrain, Ytrain = X[:Ntrain], Y[:Ntrain]
Xtest, Ytest = X[Ntrain:], Y[Ntrain:]
Xtrain.shape, Ytrain.shape

((4408, 784), (4408,))

In [113]:
model = DecisionTree() 

t0 = datetime.now() 
model.fit(Xtrain, Ytrain) 
print("Train Time:", (datetime.now()-t0))

t0 = datetime.now() 
print("Train Accuracy:", model.score(Xtrain, Ytrain))
print("Time to compute train accuracy", (datetime.now() - t0))

t0 = datetime.now() 
print("Test Accuracy:", model.score(Xtest, Ytest))
print("Time to compute test accuracy", (datetime.now()-t0))


Train Time: 0:00:25.721000
Train Accuracy: 1.0
Time to compute train accuracy 0:00:00.010000
Test Accuracy: 0.9945553539019963
Time to compute test accuracy 0:00:00.010000


In [115]:
# Maximum depth used during fit
model.max_depth_used_

7